In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot as plt
%matplotlib inline

# Importing Deep Learning Libraries

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from keras.optimizers import Adam,SGD,RMSprop

In [ ]:
picture_size = 48     # in pixel 48x48 pixel
folder_path = "/kaggle/input/face-expression-recognition-dataset/images"

In [ ]:
expression = 'angry'
plt.figure(figsize= (12,12))
for i in range(1, 10, 1):
    plt.subplot(3,3,i)
    img = load_img(folder_path+"/train/"+expression+"/"+
                  os.listdir(folder_path + "/train/" + expression)[i], target_size=(picture_size, picture_size))
    plt.imshow(img)
plt.show()

In [ ]:
batch_size  = 64

datagen_train  = ImageDataGenerator()
datagen_val = ImageDataGenerator()

train_set = datagen_train.flow_from_directory(folder_path+"/train/",
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical')

test_set = datagen_val.flow_from_directory( folder_path+"/validation/",
                                            target_size = (picture_size,picture_size),
                                            color_mode = "grayscale",
                                            batch_size=batch_size,
                                            class_mode='categorical')

In [ ]:
no_of_classes = 7

model = Sequential()

#1st CNN layer
model.add(Conv2D(32 ,kernel_size=(3, 3), activation='relu',input_shape = (48,48,1)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#2nd CNN Layer
model.add(Conv2D(128 ,kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#Flattening the layers:
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

#Compiling the layer:
opti = Adam(learning_rate = 0.0001)
model.compile(optimizer=opti,loss='categorical_crossentropy', metrics=['accuracy'])

#Printing the summary:
model.summary()

In [ ]:
#Training the Model:
model_info = model.fit(
    train_set,
    steps_per_epoch = 28821//64,
    epochs = 100,
    validation_data = test_set,
    validation_steps = 7066//64
)

In [ ]:
#Saving the Model Structure in JSON file:
model_json = model.to_json()
with open("/kaggle/working/model_v3.json","w") as json_file:
    json_file.write(model_json)

#Save trained model weights in .h5 file:
model.save_weights("/kaggle/working/new_model_v3.h5")
model.save("/kaggle/working/model_v3.h5")